# NoobAI XL 1.1 - Text-to-Image Generation with Customizable Parameters

This notebook allows you to use the [NoobAI XL 1.1](https://huggingface.co/Laxhar/noobai-XL-1.1) model for text-to-image generation on [Google Colab](https://colab.research.google.com/). You can customize various parameters such as sampler methods, sampling steps, seeds, CFG scale, image size, and apply high-resolution fixes.

In [ ]:
# Install necessary libraries
!pip install torch==1.13.1+cu118 torchvision==0.14.1+cu118 torchaudio==0.13.1+cu118 --extra-index-url https://download.pytorch.org/whl/cu118
!pip install diffusers transformers accelerate scipy ftfy
!pip install xformers

In [ ]:
# Import libraries
import torch
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
from transformers import CLIPTokenizer
import matplotlib.pyplot as plt
from PIL import Image

## Load the Model with Customized Tokenizer Configuration

We will modify the tokenizer configuration to remove the character limit by setting a high `max_length`.

In [ ]:
# Load and modify tokenizer configurations
from transformers import CLIPTokenizer

# Function to load and modify tokenizer
def load_modified_tokenizer(model_name, new_max_length=1000):
    tokenizer = CLIPTokenizer.from_pretrained(model_name, use_fast=True)
    tokenizer.model_max_length = new_max_length
    return tokenizer

# Load both tokenizers with increased max_length
tokenizer_1 = load_modified_tokenizer("Laxhar/noobai-XL-1.1")
tokenizer_2 = load_modified_tokenizer("Laxhar/noobai-XL-1.1")

## Initialize the Pipeline with Custom Parameters

Set up the Stable Diffusion pipeline with customizable parameters.

In [ ]:
# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define scheduler
scheduler = EulerDiscreteScheduler.from_pretrained(
    "Laxhar/noobai-XL-1.1",
    subfolder="scheduler",
)

# Load the pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    "Laxhar/noobai-XL-1.1",
    scheduler=scheduler,
    tokenizer=tokenizer_1,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    revision="main"
)
pipe = pipe.to(device)

## Define Generation Parameters


In [ ]:
# Define default parameters
sampler_methods = ["Euler", "Heun", "DPM2", "DDIM"]
default_sampler = "Euler"
default_steps = 50
default_seed = 42
default_cfg_scale = 7.5
default_image_size = 512
default_hires_fix = True
hires_scale = 2.0
hires_steps = 50
hires_image_size = 1024

## User Inputs for Customization

In [ ]:
from ipywidgets import interact, Dropdown, IntSlider, FloatSlider, Checkbox, Text
import random

def generate_image(
    prompt,
    sampler=default_sampler,
    steps=default_steps,
    seed=default_seed,
    cfg_scale=default_cfg_scale,
    image_size=default_image_size,
    hires_fix=default_hires_fix,
    hires_scale=hires_scale,
    hires_steps=hires_steps,
    hires_image_size=hires_image_size
):
    if seed == -1:
        seed = random.randint(0, 1000000)
    generator = torch.Generator(device=device).manual_seed(seed)
    
    # Update scheduler based on sampler
    if sampler == "Euler":
        pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
    
    # Generate image
    image = pipe(
        prompt,
        num_inference_steps=steps,
        guidance_scale=cfg_scale,
        generator=generator,
        height=image_size,
        width=image_size
    ).images[0]
    
    # High-resolution fix
    if hires_fix:
        image = image.resize((hires_image_size, hires_image_size), Image.LANCZOS)
    
    plt.figure(figsize=(8,8))
    plt.imshow(image)
    plt.axis("off")
    plt.show()
    
interact(
    generate_image,
    prompt=Text(value='A futuristic cityscape at sunset', description='Prompt:'),
    sampler=Dropdown(options=sampler_methods, value=default_sampler, description='Sampler:'),
    steps=IntSlider(value=default_steps, min=10, max=100, step=10, description='Steps:'),
    seed=IntSlider(value=default_seed, min=-1, max=1000000, step=1, description='Seed (-1 for random):'),
    cfg_scale=FloatSlider(value=default_cfg_scale, min=1.0, max=20.0, step=0.5, description='CFG Scale:'),
    image_size=IntSlider(value=default_image_size, min=256, max=1024, step=64, description='Image Size:'),
    hires_fix=Checkbox(value=default_hires_fix, description='High-Res Fix'),
    hires_scale=FloatSlider(value=hires_scale, min=1.0, max=4.0, step=0.1, description='HiRes Scale:'),
    hires_steps=IntSlider(value=hires_steps, min=10, max=100, step=10, description='HiRes Steps:'),
    hires_image_size=IntSlider(value=hires_image_size, min=512, max=2048, step=128, description='HiRes Size:')
)